# Lambda Calculus
Lambda calculus (also written as λ-calculus) is a formal system in mathematical logic for expressing computation based on function abstraction and application using variable binding and substitution. It is a universal model of computation that can be used to simulate any Turing machine. It was first introduced by mathematician Alonzo Church in the 1930s as part of his research of the foundations of mathematics [(wikipedia)](https://en.wikipedia.org/wiki/Lambda_calculus).

## Detour: Python Lambdas